In [90]:
import numpy as np

In [1]:
#Function using the log-sum-exp trick#
def logSumExp(a):
    b = np.max(a)
    return(b + np.log(np.sum(np.exp(a-b))))

#####################
##Forward Algorithm##
#####################

#Function to run forward algorithm, arguments are n = # obs, m = # states for z,#
#k = # states for x, pi = initial distribution(m vector), 
#Tmat = transition matrix (mxm), phi = emission distribution (m x k matrix)#
#x is the observed data#
#takes log of pi, Tmat and phi

def forwardAlg(n, m, k, pi, Tmat, phi, x):
    g = np.zeros((n,m))
    for i in range(0,m):
        g[0,i] = (pi[i]) + (phi[i, x[0]])
    
    for j in range(1, n):
        for l in range(0, m):
            g[j,l] = logSumExp(g[j-1, :]+(Tmat[:,l])+(phi[l,x[j]-1]))
    return(g)

def pForward(g, x):
    pXf = logSumExp(g[len(x)-1,:])
    return(pXf)

In [52]:
x = np.loadtxt('x.txt')
xNew = x - 1
m = 10
n = len(x)
k = 59
pi = np.full(m, 1/m)
phi = np.full((m,k), 1/k)
Tmat = np.full((m,m), 1/m)

In [53]:
g = forwardAlg(n,m,k,np.log(pi),np.log(Tmat),np.log(phi),xNew)
pXf = pForward(g,xNew)
pXf

/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:19: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:23: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


-1190.640933620467

In [54]:
def backwardAlg(n, m, k, pi, Tmat, phi, x):
    r = np.zeros((n,m))
    for j in range(n-2, -1, -1):
        for l in range(0, m):
            r[j, l] = logSumExp(r[j+1,: ] + Tmat[l,:] + phi[:, x[j+1]])
    
    return(r)

#Function to return p(x_1:n) from matrix from backward algorithm
def pBackward(r, pi, phi, x):
    pXb = logSumExp(r[0,: ]+ pi +phi[:,x[0]])
    return(pXb)

In [55]:
r = backwardAlg(n, m, k, np.log(pi), np.log(Tmat), np.log(phi), xNew)
pBackward(r, np.log(pi), np.log(phi), xNew)

/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


-1190.640933620467

In [83]:
def BaumWelch(n, m, k, x, tol):
    #randomly initialize pi, phi and T#
    vals = np.random.rand(m)
    pi = np.log(vals/np.sum(vals))
    Tmat = np.zeros(shape = (m, m))
    phi = np.zeros(shape = (m, k))
    for i in range(0, m):
        vals1 = np.random.rand(m)
        Tmat[i, ] = np.log(vals1/np.sum(vals1))
        vals2 = np.random.rand(k)
        phi[i, ] = np.log(vals2/np.sum(vals2))
    
    iterations = 0
    convergence = 0
    pOld = 1E10
    
    #Initialize matricies for gamma and beta values#
    gamma = np.zeros(shape = (n, m))
    beta = np.zeros(shape = (n,m,m))
    
    #Stop iterations when log(p(x_1:n)) differs by tol between iterations#
    while convergence == 0:
        #Perform forward and backward algorithms# 
        g = forwardAlg(n, m, k, pi, Tmat, phi, x)
        h = backwardAlg(n, m, k, pi, Tmat, phi, x)
        pNew = pForward(g, x)
        
        ##E-Step##
    
        #Calculate gamma and beta#
        for t in range(0, n):
            gamma[t,] = g[t,] + h[t,] - pNew
        for t in range(0, n):
            for i in range(0, m):
                for j in range(0, m):
                    if t == 1:
                        beta[t,i,j] = 1
                    else:
                        beta[t,i,j] = Tmat[i,j] + phi[j, x[t]] + g[t-1, i] + h[t, j] - pNew
        ##M-Step##
    
        #Update pi, phi and Tmat#
        pi = gamma[0,] - logSumExp(gamma[0,])
        for i in range(0, m):
            for j in range(0, m):
                Tmat[i,j] = logSumExp(beta[range(1, n), i, j]) - logSumExp(beta[range(1,n), i, ])
        for i in range(0,m):
            for w in range(0, k):
                indicies = np.where(x == w)
                phi[i,w] = logSumExp(gamma[indicies, i]) - logSumExp(gamma[:,i])
        
        criteria = abs(pOld - pNew)
        if criteria < tol:
            convergence = 1
        else:
            convergence = 0
            pOld = pNew
            iterations +=1
        return (iterations, pNew, np.exp(pi), np.exp(phi), np.exp(Tmat))
        

In [117]:
it, p, pi, phi, Tmat = BaumWelch(n, 100, k, xNew, 1)

/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:19: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:23: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:39: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [113]:
def decode(x, code):
    output = " "
    for i in range(0, len(x)):
        output = output + " " + code[x[i]] 
    return output

In [102]:
codeTest = pd.read_table('code.txt', delimiter = " ", header = None)
words = codeTest.ix[:,1]

In [107]:
words

0             .
1             A
2           The
3             a
4           air
5           and
6            at
7           ate
8          ball
9        banana
10          big
11         bird
12          bit
13        black
14         blue
15        brown
16          cat
17       chased
18      climbed
19         cute
20    delicious
21          dog
22          fat
23         fish
24     friendly
25        green
26          had
27          hat
28        house
29       hungry
30           in
31       jumped
32       little
33         long
34          man
35         mean
36       monkey
37           on
38       orange
39       purple
40          ran
41     sandwich
42          sat
43          saw
44        shirt
45        short
46        table
47         tail
48         tall
49          the
50        threw
51        tiger
52         tree
53         ugly
54        under
55          was
56         with
57        woman
58       yellow
Name: 1, dtype: object

In [114]:
decode(xNew, words)

/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: FutureWarning: scalar indexers for index type Int64Index should be integers and not floating point


'  A little cute monkey was in a big green tree . A delicious yellow banana was in the tree . The little monkey ate the banana . A short man with a brown ugly hat and a tall woman with a blue shirt sat at a table . The woman had a cute little dog with a short tail . The man ate a big delicious sandwich . The dog ate the ugly hat under the table . A big ugly dog chased a little yellow cat . The cat climbed a tree . The dog sat under the tree . A woman saw the dog under the tree and threw a ball in the air . The dog chased the ball and the cat ran . A man had a little blue house . The man had a dog and a mean hungry cat and a purple fish . The cat ate the delicious fish . A big sandwich was on the table . The dog climbed the table and ate the sandwich . A short woman had a friendly monkey with a long brown tail and a little blue hat . The monkey climbed a tall tree and threw a green banana . The woman chased the banana . A mean orange tiger saw a little man and chased the man . A little 

In [115]:
def hmm(n, pi, phi, Tmat, code):
    m = Tmat.shape[0]
    k = phi.shape[1]
    zstates = range(0, m)
    xstates = range(0, k)
    z = np.zeros(n)
    x = np.zeros(n)
    z[0] = np.random.choice(zstates, size = 1, p = pi)
    for j in range(1, n):
        z[j] = np.random.choice(zstates, size = 1, p = Tmat[z[j-1], :])
    for i in range(0, n):
        x[i] = np.random.choice(xstates, size = 1, p = phi[z[i], :])
    output = decode(x, code)
    return output


In [118]:
hmm(n, pi, phi, Tmat, words)

/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:12: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: FutureWarning: scalar indexers for index type Int64Index should be integers and not floating point


'  delicious man . in brown tree little little tail a hat man a The cute hat . and and A monkey green and . at . woman big dog ugly and . ate saw short table saw little sandwich friendly had The chased big blue banana at a under brown under ball The green chased ugly and A yellow tree . short The dog a A delicious cute short The dog The blue a a ran . little in cute . dog threw The monkey The chased the man dog dog the tiger in the ate the chased woman tree . and delicious banana a the threw was banana tree banana big banana tail cat was had A . shirt climbed shirt climbed ugly little The The a green in tail a dog big . had was ate the climbed fish The . cat ate man a A ate man tail ran brown A with monkey ball threw ate dog sandwich tree with The sat yellow monkey big dog dog ran ate green sat in a woman ugly ate with tree cute saw cute . big dog ugly yellow brown dog woman . cat ate tree a The bit in under yellow A . green cute banana with The monkey brown The monkey woman The monkey